In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl

from bin import *

from IPython.display import clear_output
%load_ext autoreload
%autoreload 2

In [2]:
mpl.use("pgf")

mpl.rcParams.update({
    "pgf.texsystem": "lualatex",
    'font.family': 'serif',
    'font.size' : 8,
    'text.usetex': True,
    'pgf.rcfonts': False,
    'pgf.preamble': tex_preamble,
})

In [3]:
n = 100
Λ = model_problem_spectrum(n,.6,1e3)
λmin = np.min(Λ)
λmax = np.max(Λ)

A = np.diag(Λ)

v = np.ones(n)
v /= np.linalg.norm(v)

f = lambda x: 1/x

fAv = np.diag(f(Λ))@v

In [4]:
k_max = 44
Q,(α,β) = lanczos_reorth(A,v,k_max,reorth=0)

err_FA = np.full(k_max,np.nan)
for k in range(k_max):
    err_FA[k] = np.linalg.norm(np.diag(np.sqrt(Λ))@(fAv - lanczos_FA(f,Q,α,β,k)))/np.linalg.norm(np.diag(np.sqrt(Λ))@v)

In [5]:
k_max = 23
Q_ro,(α_ro,β_ro) = lanczos_reorth(A,v,k_max,reorth=k_max)

err_FA_ro = np.full(k_max,np.nan)
for k in range(k_max):
    err_FA_ro[k] = np.linalg.norm(np.diag(np.sqrt(Λ))@(fAv - lanczos_FA(f,Q_ro,α_ro,β_ro,k)))/np.linalg.norm(np.diag(np.sqrt(Λ))@v)

In [6]:
k_max = 44
err_ℐ = np.full(k_max,np.nan)
ℐ = np.linspace(λmin,λmax,2000)

i=[0]
for k in range(k_max):
    i_add = np.argmax(np.min(np.abs(ℐ[i]-ℐ[:,None]),axis=1))
    i = np.append(i,i_add)
    i = np.sort(i)
    i,p,ub,lb = remez(f,ℐ,k,i=i,tol=1e-12,max_iter=300)
    err_ℐ[k] = ub

In [7]:
k_max = 23
err_Λ = np.full(k_max,np.nan)

i=[1]
for k in range(k_max):
    i_add = np.argmax(np.min(np.abs(Λ[i]-Λ[:,None]),axis=1))
    i = np.append(i,i_add)
    i = np.sort(i)
    i,p,ub,lb = remez(f,Λ,k,i=i,tol=1e-10,max_iter=300)
    err_Λ[k] = ub

/home/tyler/Documents/GitHub/Research_Code/thesis/code/bin/remez.py:37: UserWarning: failed to converge to specified tollerence for degree 2
  warnings.warn(f"failed to converge to specified tollerence for degree {k}")
/home/tyler/Documents/GitHub/Research_Code/thesis/code/bin/remez.py:37: UserWarning: failed to converge to specified tollerence for degree 14
  warnings.warn(f"failed to converge to specified tollerence for degree {k}")
/home/tyler/Documents/GitHub/Research_Code/thesis/code/bin/remez.py:37: UserWarning: failed to converge to specified tollerence for degree 15
  warnings.warn(f"failed to converge to specified tollerence for degree {k}")
/home/tyler/Documents/GitHub/Research_Code/thesis/code/bin/remez.py:37: UserWarning: failed to converge to specified tollerence for degree 16
  warnings.warn(f"failed to converge to specified tollerence for degree {k}")
/home/tyler/Documents/GitHub/Research_Code/thesis/code/bin/remez.py:37: UserWarning: failed to converge to specified toll

In [8]:
fig = plt.figure(figsize=(figure_width*cm,figure_height*cm))

ax = fig.add_axes([left, bottom, width, height])

ax.plot(err_Λ,**line_styles['l4'],label='bound on $\Lambda$')
ax.plot(err_ℐ,**line_styles['l3'],label=r'bound on $\mathcal{I}$')
ax.plot(err_FA_ro,**line_styles['l2'],label='reorth')
ax.plot(err_FA,**line_styles['l1'],label='no reorth')

ax.grid(True,linestyle=':',linewidth=.5)

ax.set_xlabel('number of matvecs: $k$')
ax.set_yscale('log')

#plt.savefig('imgs/ch1_unif_vs_spec.pgf')
plt.savefig('imgs/ch1_unif_vs_spec.pdf')
plt.close()